In [ ]:
'''
Corrective RAG Implementation
Author: Christian Sarmiento
Purpose: This notebook is intended to get a Self-RAG implementation set up with LangChain/LangGraph.
Date Created: 12/4/24
Last Updated: 12/4/24
Data: Marist College Administrative Corpus Dataset
Sources:
- https://blog.langchain.dev/agentic-rag-with-langgraph/
- https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_crag.ipynb?ref=blog.langchain.dev
Note: Most of the code for graph implementation of CRAG was taken from the second source.
-----------------------------------------------------------------------------------------------------------------------
RAG Research             |               Machine Learning Independent Study             |              DR. EITEL LAURIA
'''

In [1]:
%pip install tavily-python

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 26.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [50]:
pip install -qU openpyxl

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Imports
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from typing_extensions import TypedDict
from langgraph.graph import END, StateGraph, START
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools import DuckDuckGoSearchResults
from langchain.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic


import time
import asyncio
from queue import Queue
import threading 
import pandas as pd
import os
import gradio as gr  # easy frontend implementation
from pprint import pprint
import numpy as np
import json
from typing import List, Dict, Optional, Tuple
import sys
sys.path.append("/Users/christiansarmiento/Library/CloudStorage/OneDrive-MaristCollege/Machine Learning/Private Code")
from api_keys import openAIKey
from api_keys import langchainKey
from api_keys import tavilyKey  # web search packages
from api_keys import anthropicKey


from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity, LLMContextPrecisionWithoutReference
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas import SingleTurnSample

/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm 

In [2]:
# LangChain Enviornment Variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = langchainKey()
os.environ["OPENAI_API_KEY"] = openAIKey()
os.environ["TAVILY_API_KEY"] = tavilyKey()

In [24]:
# Load Data
csvPath = "/Users/christiansarmiento/Library/CloudStorage/OneDrive-MaristCollege/Machine Learning/Data/Marist_QA.csv"
maristQA = pd.read_csv(csvPath, header=None)

# To use RecursiveCharacterTextSplitter, we need a list of dictionaries
maristContext = [Document(page_content=text) for text in maristQA[1].tolist()]

In [25]:
# Split Documents into Chunks
textSplitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
texts = textSplitter.split_documents(maristContext)

In [26]:
# Store Documents in Vector DB (Chroma)
vectorDB = Chroma.from_documents(documents=texts, embedding=OpenAIEmbeddings())

# Setup Retrieval System
retriever = vectorDB.as_retriever(search_type="similarity", search_kwargs={"k": 3})  # Retrieves 3 documents

In [27]:
# Define Grader class for document grading in CRAG
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

# LLM with function call
llmDocGrader = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structuredLLMGrader = llmDocGrader.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
   tic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
gradePrompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

# Grader Chain
retrievalGrader = gradePrompt | structuredLLMGrader

# Testing it
question = "agent memory"
docs = retriever.get_relevant_documents(question)
docTxt = docs[1].page_content
print(retrievalGrader.invoke({"question": question, "document": docTxt}))

binary_score='no'


In [28]:
# Generation Chain

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
ragChain = prompt | llm | StrOutputParser()

# Run
generation = ragChain.invoke({"context": docs, "question": question})
print(generation)

I don't know.


In [29]:
# Question Rewriter
llmRewriter = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
rewritePrompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

questionRewriter = rewritePrompt | llm | StrOutputParser()
questionRewriter.invoke({"question": question})

'What are the key concepts and techniques related to agent memory in artificial intelligence?'

In [30]:
# Web Search Component
webSearchTool = TavilySearchResults(k=3)

In [31]:
# Define metric evaluator

## Evaluation LLM & embeddings
evalLLM = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))
evalEmbeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

## Initialize metrics with LLM and embeddings
contextRecall = LLMContextRecall(llm=evalLLM)
faithfulness = Faithfulness(llm=evalLLM)
factualCorrectness = FactualCorrectness(llm=evalLLM)
semanticSimilarity = SemanticSimilarity(embeddings=evalEmbeddings)

## Collect metrics
evalMetrics = [
    contextRecall,
    faithfulness,
    factualCorrectness,
    semanticSimilarity
]

In [41]:
# Define Graph structure for Self-RAG

# Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
        web_search: whether to add search
        metrics: evaluation metrics for each generation 
    """
    question: str
    generation: str
    webSearch: str
    documents: List[str]
    metrics: Optional[Dict[str, float]]
    visitedInternet: bool

## Nodes

# Retrieval Node     
def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question, "visitedInternet": state["visitedInternet"]}


# Generation Node
def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = ragChain.invoke({"context": documents, "question": question})

    # Return updated state
    updatedState = {"documents": documents, "question": question, "generation": generation, "visitedInternet": state["visitedInternet"]}

    return updatedState

# Grader Node
def gradeDocuments(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filteredDocs = []
    webSearch = "No"
    for d in documents:

        score = retrievalGrader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filteredDocs.append(d)

        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            webSearch = "Yes"
            continue

    return {"documents": filteredDocs, "question": question, "webSearch": webSearch, "visitedInternet": state["visitedInternet"]}


# Rewriter node
def transformQuery(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    betterQuestion = questionRewriter.invoke({"question": question})
    return {"documents": documents, "question": betterQuestion, "visitedInternet": state["visitedInternet"]}

def webSearch(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]
    internetFlag = state["visitedInternet"]

    # Web search
    docs = webSearchTool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)
    internetFlag = True

    return {"documents": documents, "question": question, "visitedInternet": internetFlag}

## Edges

# Generation edge
def decideToGenerate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    webSearch = state["webSearch"]
    state["documents"]

    if webSearch == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transformQuery"
    
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"
    
async def evaluateMetrics(state):
    """
    Evaluate metrics for the current RAG pipeline response.

    Args:
        state (dict): The current graph state.

    Returns:
        state (dict): Adds a 'metrics' key containing evaluation scores.
    """
    print("---EVALUATING METRICS---")
    question = state["question"]
    generation = state["generation"]
    documents = state["documents"]

    # Mock ground truth if unavailable (replace with actual reference if possible)
    groundTruth = state.get("groundTruth", "Expected answer based on context.")

    # Prepare retrieved contexts
    retrievedContexts = [doc.page_content for doc in documents]

    # Create a SingleTurnSample object
    sample = SingleTurnSample(
        user_input=question,
        response=generation,
        reference=groundTruth,
        retrieved_contexts=retrievedContexts,
    )

    # Evaluate metrics
    state["metrics"] = {
        "LLMContextRecall": await contextRecall.single_turn_ascore(sample),
        "Faithfulness": await faithfulness.single_turn_ascore(sample),
        "FactualCorrectness": await factualCorrectness.single_turn_ascore(sample),
        "SemanticSimilarity": await semanticSimilarity.single_turn_ascore(sample),
    }

    return state

In [42]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  
workflow.add_node("gradeDocuments", gradeDocuments)  
workflow.add_node("generate", generate)  
workflow.add_node("transformQuery", transformQuery)  
workflow.add_node("webSearchNode", webSearch)
workflow.add_node("evaluateMetrics", evaluateMetrics)  

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "gradeDocuments")
workflow.add_conditional_edges(
    "gradeDocuments",
    decideToGenerate,
    {
        "transformQuery": "transformQuery",
        "generate": "generate",
    },
)
workflow.add_edge("transformQuery", "webSearchNode")
workflow.add_edge("webSearchNode", "generate")
workflow.add_edge("generate", "evaluateMetrics")
workflow.add_edge("evaluateMetrics", END)

# Compile
app = workflow.compile()

In [16]:
# Inital test run
inputs = {"question": "Who is Carolyn Matheus?"}
async for output in app.astream(inputs):
    for key, value in output.items():

        # Print node
        pprint(f"Node '{key}':")

        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)

        # Print metrics
        if "metrics" in value:
            pprint("Metrics: ")
            pprint(value["metrics"])

    pprint("\n---\n")

# Final generation
pprint("Final Generation: ")
pprint(value["generation"])

# Final metrics
if "metrics" in value:
    pprint("Final Metrics: ")
    pprint(value["metrics"])

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'gradeDocuments':"
'\n---\n'
---GENERATE---
"Node 'generate':"
'\n---\n'
---EVALUATING METRICS---
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7259663840810067}
'\n---\n'
'Final Generation: '
('Dr. Carolyn C. Matheus is an Associate Professor of Information Systems and '
 'the Director of the Honors Program at Marist College. She holds a PhD in '
 'Organizational Studies with a focus on leadership from SUNY Albany and has '
 'received the National Society of Leadership and Success award for Excellence '
 'in Teaching. Dr. Matheus is involved in faculty-student research projects '
 'and offers seminars on authentic leadership and innovation.')
'Final Metrics

In [65]:
# Put CRAG into Gradio
evaluationSamples = []
ragResults = []
async def correctiveRAG(userQuery, history, correctAnswer=None):
    """
    Gradio-compatible function to process CRAG workflow.
    Args:
        userQuery (str): The user's question.
        history (list): Conversation history.
        correctAnswer (str): The ground truth answer for metrics (optional).

    Returns:
        tuple: (chatDisplay, history)
    """

    # Variables
    inputs = {"question": userQuery, "visitedInternet": False}
    finalOutput = None
    internetFlag = False

    # Start the workflow
    async for output in app.astream(inputs):

        # Saving final output for metric purposes
        finalOutput = output

        # Check if the process went to the internet
        if "webSearchNode" in output:
            internetFlag = True

        # Printing out each node state for clarity
        for key, value in output.items():

            # Print node
            pprint(f"Node '{key}':")

            # Print metrics
            if "metrics" in value:
                pprint("Metrics: ")
                pprint(value["metrics"])
    
    # Get the generation and its metrics
    finalNodeKey = list(finalOutput.keys())[-1]  # Get the key of the last executed node
    nodeOutput = finalOutput[finalNodeKey]  # Access the nested state
    generation = nodeOutput.get("generation", "No generation produced.")
    metrics = nodeOutput.get("metrics", {})

    # Update history
    if history is None:
        history = []
    
    history.extend([
        {"role": "user", "content": userQuery},
        {"role": "llm", "content": generation}
    ])

    # Display output for gradio
    #chatDisplay = [(msg["content"], "User" if msg["role"] == "user" else "LLM") for msg in history]

    # Append metrics to evaluationSamples for tracking (if correctAnswer is provided)
    if correctAnswer:
        evaluationSamples.append({
            "user_input": userQuery,
            "retrieved_contexts": [doc.page_content for doc in finalOutput.get("documents", [])],
            "response": generation,
            "reference": correctAnswer,
            "metrics": metrics,
        })
    
    # Save the result of the query with the metrics 
    ragResults.append({
        "question": userQuery,
        "generation": generation,
        "retry": internetFlag
    })

    return history #, chatDisplay

In [ ]:
# Gradio frontend
interface = gr.Interface(
    fn=correctiveRAG,
    inputs=[
        gr.Textbox(label="Ask a Question", placeholder="Enter your question here..."),
        gr.State(),  # Keeps track of conversation history
        gr.Textbox(label="Correct Answer (Optional)", placeholder="For evaluation purposes..."),
    ],
    outputs=[
        gr.Chatbot(label="CorrectiveRAG Conversation"),
        gr.State(),  # Updates conversation history
    ],
    title="CorrectiveRAG Implementation",
    description="Interact with the CRAG workflow for document-grounded question answering.",
)

# Launch the interface
interface.launch()

/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/gradio/components/chatbot.py:222: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'gradeDocuments':"
---GENERATE---
"Node 'generate':"
---EVALUATING METRICS---
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7259663840810067}


In [61]:
# Function to evaluate our RAG pipeline when given ground truth
async def pipelineEvaluation(dataset, metrics):

    # Run through our runs
    results = []
    for run in dataset:

        # Save our inputs/outputs
        inputQuery = run["user_input"]
        groundTruthAnswer = run["reference"]
        contexts = run["retrieved_contexts"]
        response = run["response"]

        # Create a SingleTurnSample object
        sample = SingleTurnSample(
            user_input=inputQuery,
            response=response,
            reference=groundTruthAnswer,
            retrieved_contexts=contexts 
        )

        # Evaluate metrics
        runResults = {"question": inputQuery}
        for metric in metrics:

            # Get the score for the given metric
            try:

                score = await metric.single_turn_ascore(sample)
                runResults[type(metric).__name__] = score

            except Exception as e:
                # Catch errors for debugging
                runResults[type(metric).__name__] = f"Error: {str(e)}"
        
        # Save metric results
        results.append(runResults)
    
    # Calculate mean and standard deviation for each metric
    metricsStats = {}
    for metric in metrics:
        metricName = type(metric).__name__
        scores = [result[metricName] for result in results if isinstance(result[metricName], (int, float))]
        
        # Only calculate stats if there are valid scores
        if scores:
            metricsStats[metricName] = {
                "mean": np.mean(scores),
                "std_dev": np.std(scores),
            }
            
        else:
            metricsStats[metricName] = {
                "mean": "No valid scores",
                "std_dev": "No valid scores",
            }
    
    return results, metricsStats

In [44]:
# Load metrics
evalMetrics = [LLMContextRecall(llm=LangchainLLMWrapper(llm)), 
               FactualCorrectness(llm=LangchainLLMWrapper(llm)), 
               Faithfulness(llm=LangchainLLMWrapper(llm)), 
               SemanticSimilarity(embeddings=LangchainEmbeddingsWrapper(OpenAIEmbeddings()))]

In [21]:
# Evaluate our pipeline responses
evalResults = await pipelineEvaluation(evaluationSamples, evalMetrics)
for result in evalResults:
    print(result)

{'input_query': 'Who is Carolyn Matheus?', 'LLMContextRecall': 0.0, 'FactualCorrectness': 0.0, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.928909234587568}


In [74]:
# Sample 222 records from our dataset
maristTestSample = maristQA.sample(100, replace=False)
maristTestSample.head()

,0,1
586,How to avoid spams,"""\u00a0Impersonation emails\u00a0Robocalls cla..."
587,What is the phone number for student financial...,"""Transfer Student Admission Department Student..."
568,What happens to the funds in my Marist Money a...,"""Student Services Department Card Services Nam..."
103,When is the the office of financial services o...,"""Job Location and Development Information (Fox..."
105,tell me about marist counseling,"""Counseling Services Department Counseling Ser..."


In [ ]:
# Run our chain with each question and evaluate
chatHistory = None
for row in maristTestSample.iterrows():
    chatHistory = await correctiveRAG(row[1][0], chatHistory, row[1][1])

## Evaluation
evalResults, metricStats = await pipelineEvaluation(evaluationSamples, evalMetrics)
for result in evalResults:
    print(result)

for metric in metricStats.keys():
    print(f"{metric} - Mean: {metricStats[metric]['mean']}, St. Dev: {metricStats[metric]['std_dev']}")

---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'gradeDocuments':"
---GENERATE---
"Node 'generate':"
---EVALUATING METRICS---
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7596856019922399}
---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'gradeDocuments':"
---GENERATE---
"Node 'generate':"
---EVALUATING METRICS---
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7045918562747744}
---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUE

The LLM did not return a valid classification.


"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': nan,
 'SemanticSimilarity': 0.7383440103663941}
---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---
"Node 'gradeDocuments':"
---TRANSFORM QUERY---
"Node 'transformQuery':"
---WEB SEARCH---
"Node 'webSearchNode':"
---GENERATE---
"Node 'generate':"
---EVALUATING METRICS---
"Node 'evaluateMetrics':"
'Metrics: '
{'FactualCorrectness': 0.0,
 'Faithfulness': 1.0,
 'LLMContextRecall': 0.0,
 'SemanticSimilarity': 0.7349868099610156}
---RETRIEVE---
"Node 'retrieve':"
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL D

Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=5dd6db36-90e3-4207-b4ac-c7d41cf5ba24,id=5dd6db36-90e3-4207-b4ac-c7d41cf5ba24; trace=5dd6db36-90e3-4207-b4ac-c7d41cf5ba24,id=5589d19a-9bc9-485e-bc90-ca775f49d5d7; trace=5dd6db36-90e3-4207-b4ac-c7d41cf5ba24,id=61a27edd-012b-497d-ac66-b1096d2dbae1; patch: trace=dabfbcec-c8bb-43c0-82d7-def740475b96,id=cbe9cc23-f9c6-4b37-98e9-7be1960274a6; trace=dabfbcec-c8bb-43c0-82d7-def740475b96,id=dabfbcec-c8bb-43c0-82d7-def740475b96; trace=dabfbcec-c8bb-43c0-82d7-def740475b96,id=b1101fbd-dbfb-4827-9b48-019d595c0283
Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests f

{'question': 'What GPA do I need for grad school?', 'LLMContextRecall': 1.0, 'FactualCorrectness': 0.11, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.8139424641650321}
{'question': 'Who published Displacements and Transformations in Caribbean Cultures?', 'LLMContextRecall': 0.16666666666666666, 'FactualCorrectness': 0.14, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.8410372116485767}
{'question': 'Who is the chair of the english department?', 'LLMContextRecall': 0.16666666666666666, 'FactualCorrectness': 0.0, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.793254676322769}
{'question': 'What are some of the academic resources I can leverage?', 'LLMContextRecall': 0.0, 'FactualCorrectness': 0.74, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.8964862653975958}
{'question': 'Andrew Tokash Classes?', 'LLMContextRecall': 0.0, 'FactualCorrectness': 0.07, 'Faithfulness': 0.0, 'SemanticSimilarity': 0.8689408453581797}
{'question': "Who's the person to go to for Latin American and Caribbean Studies

Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec; trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=a6fa4e94-d088-49e5-b7ac-e8fbd6522ee9; trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=5f062a7b-982d-41fd-88ec-f7d7cf611cad; trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=44007c99-0a77-4b1a-9c62-66257e6e8d80; trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=870fcd51-3f00-48b9-97d9-ee86222ff501; trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=7daee0f7-541e-447d-8eb1-be2e3ed8b89a; trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=dd80434a-c0ad-4757-9a11-c87f717ff589; trace=e568f6a0-bbcd-45cd-9cbb-12462a59b0ec,id=06968678-e7f0-42ca-992b-db64dac1144c; trace=e568f6a0-bbcd-45cd

In [76]:
# Make our dataframe of results
processResults = pd.DataFrame(ragResults)
evalResultsDF = pd.DataFrame(evalResults)
finalResultsDF = processResults.merge(evalResultsDF, on="question", how="left")
finalResultsDF

,question,generation,retry,LLMContextRecall,FactualCorrectness,Faithfulness,SemanticSimilarity
0,What GPA do I need for grad school?,You typically need a minimum overall GPA of 3....,False,1.000000,0.11,0.0,0.813942
1,What GPA do I need for grad school?,You typically need a minimum overall GPA of 3....,False,0.500000,0.10,0.0,0.813942
2,Who published Displacements and Transformation...,"The publisher of the book ""Displacements and T...",True,0.166667,0.14,0.0,0.841037
3,Who is the chair of the english department?,The current chair of the English department is...,True,0.166667,0.00,0.0,0.793255
4,What are some of the academic resources I can ...,"You can leverage several academic resources, i...",False,0.000000,0.74,0.0,0.896486
...,...,...,...,...,...,...,...
173,Where is the advising and academic services ce...,The advising and academic services center is l...,False,0.000000,0.29,0.0,0.876235
174,How to use iLearn,"To use iLearn, start by watching the short ""On...",False,0.000000,0.50,0.0,0.911958
175,Janine Peterson job?,Janine Peterson currently works at the Nationa...,True,1.000000,0.00,0.0,0.806813
176,What about Blackridge technology,Blackridge technology focuses on cybersecurity...,True,0.200000,0.00,0.0,0.727944


In [77]:
# Averages & St. devs
print(f"LLM Context Recall - Mean: {finalResultsDF["LLMContextRecall"].mean()}, St. Dev: {finalResultsDF["LLMContextRecall"].std()}")
print(f"FactualCorrectness - Mean: {finalResultsDF["FactualCorrectness"].mean()}, St. Dev: {finalResultsDF["FactualCorrectness"].std()}")
print(f"Faithfulness - Mean: {finalResultsDF["Faithfulness"].mean()}, St. Dev: {finalResultsDF["Faithfulness"].std()}")
print(f"SemanticSimilarity - Mean: {finalResultsDF["SemanticSimilarity"].mean()}, St. Dev: {finalResultsDF["SemanticSimilarity"].std()}")


LLM Context Recall - Mean: 0.3502828980522305, St. Dev: 0.38078152337104065
FactualCorrectness - Mean: 0.229438202247191, St. Dev: 0.21680191980388025
Faithfulness - Mean: 0.0025280898876404497, St. Dev: 0.023930497237328208
SemanticSimilarity - Mean: 0.861302026325832, St. Dev: 0.05792963836449268


In [81]:
final100ResultsDF = finalResultsDF.tail(100)

In [82]:
# Dervive generations that didn't use the internet
trueContextsDF = final100ResultsDF[final100ResultsDF["retry"] == False]
trueContextsDF.describe()

,LLMContextRecall,FactualCorrectness,Faithfulness,SemanticSimilarity
count,55.000000,55.000000,55.0,55.000000
mean,0.374025,0.356000,0.0,0.888085
std,0.395259,0.213772,0.0,0.049715
min,0.000000,0.000000,0.0,0.773360
25%,0.000000,0.195000,0.0,0.857180
50%,0.250000,0.360000,0.0,0.892533
75%,0.650000,0.500000,0.0,0.925828
max,1.000000,0.780000,0.0,0.971669


In [83]:
trueContextsDF.head(100)

,question,generation,retry,LLMContextRecall,FactualCorrectness,Faithfulness,SemanticSimilarity
78,Do I have to research something specific for m...,"No, you do not have to research something spec...",False,0.571429,0.24,0.0,0.779964
82,What is accommodations and accessibility?,Accommodations and accessibility refer to the ...,False,0.000000,0.57,0.0,0.871183
86,How many hours of clinical field experience is...,The Clinical Mental Health Counseling program ...,False,0.250000,0.30,0.0,0.884294
88,Where can you do internships,You can do internships in New York City and th...,False,1.000000,0.50,0.0,0.889595
89,Admissions contact?,You can contact the Office of Graduate Admissi...,False,0.000000,0.00,0.0,0.857182
90,School of Communications internship opportunities,The School of Communication and the Arts offer...,False,0.900000,0.35,0.0,0.919466
91,on what principles is the college's education ...,The college's education is grounded on princip...,False,0.454545,0.19,0.0,0.847089
93,How can I learn about the term abroad program?,"You can learn about the term ""abroad program"" ...",False,0.500000,0.44,0.0,0.898406
94,Dr. Nicholas Marshall,Dr. Nicholas Marshall is an Associate Professo...,False,0.000000,0.55,0.0,0.920979
95,MA clinical mental health,The Master of Arts in Clinical Mental Health C...,False,0.000000,0.48,0.0,0.928664


# Corrective RAG Process w/ LangChain

In [3]:
# Create vector DB

## Load Data
evalDataPath = "/Users/christiansarmiento/Library/CloudStorage/OneDrive-MaristCollege/Machine Learning/Data/manual_eval.csv" # used for manual evaluation
csvPath = "/Users/christiansarmiento/Library/CloudStorage/OneDrive-MaristCollege/Machine Learning/Data/Cleaned_QA.csv"
maristQA = pd.read_csv(evalDataPath, sep=";" )  # add header=None if loading regular csv (csvPath - Cleaned_QA.csv), sep=";" if evalDataPath

## To use RecursiveCharacterTextSplitter, we need a list of dictionaries
maristContext = [Document(page_content=text) for text in maristQA["ground_truth"].tolist()]

## Split Documents into Chunks
textSplitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
texts = textSplitter.split_documents(maristContext)

## Store Documents in Vector DB (Chroma)
vectorDB = Chroma.from_documents(documents=texts, embedding=OpenAIEmbeddings())

In [4]:
# Define LLM 
llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000, temperature=0)

In [5]:
# Search Tool
search = DuckDuckGoSearchResults()

In [6]:
# LLM chains for retrieval evaluator, knowledge refinement, & query rewriter

## Retrieval Evaluator
class RetrievalEvaluatorInput(BaseModel):
    relevance_score: float = Field(..., description="The relevance score of the document to the query. the score should be between 0 and 1.")

def retrieval_evaluator(query: str, document: str) -> float:
    prompt = PromptTemplate(
        input_variables=["query", "document"],
        template="On a scale from 0 to 1, how relevant is the following document to the query? Query: {query}\nDocument: {document}\nRelevance score:"
    )
    chain = prompt | llm.with_structured_output(RetrievalEvaluatorInput)
    input_variables = {"query": query, "document": document}
    result = chain.invoke(input_variables).relevance_score
    return result

## Knowledge Refinement
class KnowledgeRefinementInput(BaseModel):
    key_points: str = Field(..., description="The document to extract key information from.")

def knowledge_refinement(document: str) -> List[str]:
    prompt = PromptTemplate(
        input_variables=["document"],
        template="Extract the key information from the following document in bullet points:\n{document}\nKey points:"
    )
    chain = prompt | llm.with_structured_output(KnowledgeRefinementInput)
    input_variables = {"document": document}
    result = chain.invoke(input_variables).key_points
    return [point.strip() for point in result.split('\n') if point.strip()]

## Web Search Query Rewriter
class QueryRewriterInput(BaseModel):
    query: str = Field(..., description="The query to rewrite.")
    
def rewrite_query(query: str) -> str:
    prompt = PromptTemplate(
        input_variables=["query"],
        template="Rewrite the following query to make it more suitable for a web search:\n{query}\nRewritten query:"
    )
    chain = prompt | llm.with_structured_output(QueryRewriterInput)
    input_variables = {"query": query}
    return chain.invoke(input_variables).query.strip()

In [7]:
# Helper function to parse search results
def parse_search_results(results_string: str) -> List[Tuple[str, str]]:
    """
    Parse a JSON string of search results into a list of title-link tuples.

    Args:
        results_string (str): A JSON-formatted string containing search results.

    Returns:
        List[Tuple[str, str]]: A list of tuples, where each tuple contains the title and link of a search result.
                               If parsing fails, an empty list is returned.
    """
    try:
        # Attempt to parse the JSON string
        results = json.loads(results_string)
        # Extract and return the title and link from each result
        return [(result.get('title', 'Untitled'), result.get('link', '')) for result in results]
    except json.JSONDecodeError:
        # Handle JSON decoding errors by returning an empty list
        print("Error parsing search results. Returning empty list.")
        return []

In [8]:
# Define CRAG "nodes"
def retrieve_documents(query: str, vectorstore, k: int = 3) -> List[str]:
    """
    Retrieve documents based on a query using a FAISS index.

    Args:
        query (str): The query string to search for.
        faiss_index (FAISS): The FAISS index used for similarity search.
        k (int): The number of top documents to retrieve. Defaults to 3.

    Returns:
        List[str]: A list of the retrieved document contents.
    """
    docs = vectorstore.similarity_search(query, k=k)
    return [doc.page_content for doc in docs]

def evaluate_documents(query: str, documents: List[str]) -> List[float]:
    """
    Evaluate the relevance of documents based on a query.

    Args:
        query (str): The query string.
        documents (List[str]): A list of document contents to evaluate.

    Returns:
        List[float]: A list of relevance scores for each document.
    """
    return [retrieval_evaluator(query, doc) for doc in documents]

def perform_web_search(query: str) -> Tuple[List[str], List[Tuple[str, str]]]:
    """
    Perform a web search based on a query.

    Args:
        query (str): The query string to search for.

    Returns:
        Tuple[List[str], List[Tuple[str, str]]]: 
            - A list of refined knowledge obtained from the web search.
            - A list of tuples containing titles and links of the sources.
    """
    rewritten_query = rewrite_query(query)
    web_results = search.run(rewritten_query)
    web_knowledge = knowledge_refinement(web_results)
    sources = parse_search_results(web_results)
    return web_knowledge, sources

def generate_response(query: str, knowledge: str, sources: List[Tuple[str, str]]) -> str:
    """
    Generate a response to a query using knowledge and sources.

    Args:
        query (str): The query string.
        knowledge (str): The refined knowledge to use in the response.
        sources (List[Tuple[str, str]]): A list of tuples containing titles and links of the sources.

    Returns:
        str: The generated response.
    """
    response_prompt = PromptTemplate(
        input_variables=["query", "knowledge", "sources"],
        template="Based on the following knowledge, answer the query. Include the sources with their links (if available) at the end of your answer:\nQuery: {query}\nKnowledge: {knowledge}\nSources: {sources}\nAnswer:"
    )
    input_variables = {
        "query": query,
        "knowledge": knowledge,
        "sources": "\n".join([f"{title}: {link}" if link else title for title, link in sources])
    }
    response_chain = response_prompt | llm
    return response_chain.invoke(input_variables).content


In [56]:
# Corrective RAG Process w/ LangChain
def correctiveRAGLangChain(query: str, vectorstore) -> str:
    """
    Process a query by retrieving, evaluating, and using documents or performing a web search to generate a response.

    Args:
        query (str): The query string to process.
        vectorstore: The vector DB (Chroma) used for document retrieval.

    Returns:
        str: The generated response based on the query.
    """
    print(f"\nProcessing query: {query}")

    # Retrieve and evaluate documents
    retrieved_docs = retrieve_documents(query, vectorstore)
    eval_scores = evaluate_documents(query, retrieved_docs)
    
    print(f"\nRetrieved {len(retrieved_docs)} documents")
    print(f"Evaluation scores: {eval_scores}")

    # Determine action based on evaluation scores
    max_score = max(eval_scores)
    sources = []
    
    if max_score > 0.7:
        print("\nAction: Correct - Using retrieved document")
        best_doc = retrieved_docs[eval_scores.index(max_score)]
        final_knowledge = best_doc
        sources.append(("Retrieved document", ""))
    elif max_score < 0.3:
        print("\nAction: Incorrect - Performing web search")
        final_knowledge, sources = perform_web_search(query)
    else:
        print("\nAction: Ambiguous - Combining retrieved document and web search")
        best_doc = retrieved_docs[eval_scores.index(max_score)]
        # Refine the retrieved knowledge
        retrieved_knowledge = knowledge_refinement(best_doc)
        web_knowledge, web_sources = perform_web_search(query)
        final_knowledge = "\n".join(retrieved_knowledge + web_knowledge)
        sources = [("Retrieved document", "")] + web_sources

    print("\nFinal knowledge:")
    print(final_knowledge)
    
    print("\nSources:")
    for title, link in sources:
        print(f"{title}: {link}" if link else title)

    # Generate response
    print("\nGenerating response...")
    response = generate_response(query, final_knowledge, sources)

    print("\nResponse generated")
    return response

In [14]:
# Sample a record from our dataset
maristTestSample = maristQA.sample(1, replace=False)
maristTestSample.head()

,0,1
594,Are DMA members eligible for special tuition p...,Community Organizations Department Graduate Ad...


In [ ]:
# Make sure this works by itself first

## Run our chain with a question
for row in maristTestSample.iterrows():
    response = correctiveRAGLangChain(row[1][0], vectorDB)
    print(f"Generation: {response}")




Processing query: Are DMA members eligible for special tuition pricing?

Retrieved 3 documents
Evaluation scores: [1.0, 1.0, 1.0]

Action: Correct - Using retrieved document

Final knowledge:
for innovators, artists, creators, writers, analysts, and strategists of all things related to marketing powered by data. They provide innovation that accelerates business growth and brand experiences as well as education that develops talent and insights into best practices.The DMA represents thousands of companies and nonprofit organizations that use and support data-driven marketing practices and techniques. Their members span the marketing ecosystem and have the passion for accelerating industry diversity and innovation. The Data & Marketing Association supports and drives their members to reach their fullest potential.As a way of saying \thank you\ to the many DMA members who reside in our community, Marist College is offering special tuition pricing for all members and their immediate adult

In [9]:
# Modify the function to support RAGAS evaluation
evaluationSamples = []
samplesOutputDF = pd.DataFrame(columns=["question", "ground_truth", "context", "generation", "TP", "FP", "FN"])
def correctiveRAGLangChain(query: str, vectorstore, correctAnswer: str) -> str:
    """
    Process a query by retrieving, evaluating, and using documents or performing a web search to generate a response.

    Args:
        query (str): The query string to process.
        vectorstore: The vector DB (Chroma) used for document retrieval.

    Returns:
        str: The generated response based on the query.
    """
    print(f"\nProcessing query: {query}")

    # Retrieve and evaluate documents
    retrieved_docs = retrieve_documents(query, vectorstore)
    eval_scores = evaluate_documents(query, retrieved_docs)
    
    print(f"\nRetrieved {len(retrieved_docs)} documents")
    print(f"Evaluation scores: {eval_scores}")

    # Determine action based on evaluation scores
    max_score = max(eval_scores)
    sources = []
    
    if max_score > 0.7:
        print("\nAction: Correct - Using retrieved document")
        best_doc = retrieved_docs[eval_scores.index(max_score)]
        final_knowledge = best_doc
        sources.append(("Retrieved document", ""))
    elif max_score < 0.3:
        print("\nAction: Incorrect - Performing web search")
        final_knowledge, sources = perform_web_search(query)
    else:
        print("\nAction: Ambiguous - Combining retrieved document and web search")
        best_doc = retrieved_docs[eval_scores.index(max_score)]
        # Refine the retrieved knowledge
        retrieved_knowledge = knowledge_refinement(best_doc)
        web_knowledge, web_sources = perform_web_search(query)
        final_knowledge = "\n".join(retrieved_knowledge + web_knowledge)
        sources = [("Retrieved document", "")] + web_sources

    print("\nFinal knowledge:")
    print(final_knowledge)
    
    print("\nSources:")
    for title, link in sources:
        print(f"{title}: {link}" if link else title)

    # Generate response
    print("\nGenerating response...")
    response = generate_response(query, final_knowledge, sources)

    # Save output
    if type(final_knowledge) == str:  # Needs to be in a list format for subsequent evaluation
        final_knowledge = [final_knowledge]

    if correctAnswer:
        evaluationSamples.append({
            "user_input": query,
            "retrieved_contexts": final_knowledge,
            "response": response,
            "reference": correctAnswer
        })
        samplesOutputDF.loc[len(samplesOutputDF)] = [query, correctAnswer, final_knowledge, response, -1, -1, -1]

    print("\nResponse generated")
    return response

In [74]:
# Function to evaluate our RAG pipeline when given ground truth

async def evaluateSample(sample, metrics):
    '''Helper function for pipelineEvaluation that evaluates a sample using multithreading'''

    # Evaluate metrics
    runResults = {"question": sample.user_input}
    for metric in metrics:

        # Get the score for the given metric
        try:

            print(f"Data being passed: {sample}")
            score = await metric.single_turn_ascore(sample)
            runResults[type(metric).__name__] = score
            print(f"Score for {type(metric).__name__}: {score}")

        except Exception as e:
            # Catch errors for debugging
            runResults[type(metric).__name__] = f"Error: {str(e)}"
    
    return runResults

def multithreadedEvaluation(sample, metrics, queue):
    '''Helper function that runs the async evaluation within a thread'''

    # Allows for asynchronous thread compatibility
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    results = loop.run_until_complete(evaluateSample(sample, metrics))

    queue.put(results)

async def pipelineEvaluation(dataset, metrics):
    '''Function that takes in a dataset of input and output and metrics to evaluate 
    the quality of a RAG process using RAGAS
    '''
    
    # Run through our runs
    results = []
    resultQueue = Queue()  # intermediary queue for thread data
    threads = []  
    for run in dataset:

        # Save our inputs/outputs
        inputQuery = run["user_input"]
        groundTruthAnswer = run["reference"]
        contexts = run["retrieved_contexts"]
        response = run["response"]

        # Create a SingleTurnSample object
        sample = SingleTurnSample(
            user_input=inputQuery,
            response=response,
            reference=groundTruthAnswer,
            retrieved_contexts=contexts 
        )

        # Create and start a threat for paralleization of metrics
        thread = threading.Thread(target=multithreadedEvaluation, args=(sample, metrics, resultQueue))
        threads.append(thread)
        thread.start()
    
    # Wait for the threads to finish
    for thread in threads:
        thread.join()
    
    # Get the results
    while not resultQueue.empty():
        results.append(resultQueue.get())
    
    # Calculate mean and standard deviation for each metric
    metricsStats = {}
    for metric in metrics:
        metricName = type(metric).__name__
        scores = [result[metricName] for result in results if isinstance(result[metricName], (int, float))]
        
        # Only calculate stats if there are valid scores
        if scores:
            metricsStats[metricName] = {
                "mean": np.mean(scores),
                "std_dev": np.std(scores),
            }
            
        else:
            metricsStats[metricName] = {
                "mean": "No valid scores",
                "std_dev": "No valid scores",
            }
    
    return results, metricsStats

In [10]:
# Function to evaluate our RAG pipeline when given ground truth
async def pipelineEvaluation(dataset, metrics):

    # Run through our runs
    results = []
    for run in dataset:

        # Save our inputs/outputs
        inputQuery = run["user_input"]
        groundTruthAnswer = run["reference"]
        contexts = run["retrieved_contexts"]
        response = run["response"]

        # Create a SingleTurnSample object
        sample = SingleTurnSample(
            user_input=inputQuery,
            response=response,
            reference=groundTruthAnswer,
            retrieved_contexts=contexts 
        )

        # Evaluate metrics
        runResults = {"question": inputQuery}
        for metric in metrics:

            # Get the score for the given metric
            try:

                score = await metric.single_turn_ascore(sample)
                runResults[type(metric).__name__] = score

            except Exception as e:
                # Catch errors for debugging
                runResults[type(metric).__name__] = f"Error: {str(e)}"
        
        # Save metric results
        results.append(runResults)
    
    # Calculate mean and standard deviation for each metric
    metricsStats = {}
    for metric in metrics:
        metricName = type(metric).__name__
        scores = [result[metricName] for result in results if isinstance(result[metricName], (int, float))]
        
        # Only calculate stats if there are valid scores
        if scores:
            metricsStats[metricName] = {
                "mean": np.mean(scores),
                "std_dev": np.std(scores),
            }
            
        else:
            metricsStats[metricName] = {
                "mean": "No valid scores",
                "std_dev": "No valid scores",
            }
    
    return results, metricsStats

In [11]:
# Function to clean up evaluation code
async def computeEvaluationMetrics(samples, metrics):
    '''
    Helper method to compute the metrics and its averages for a given sample. parameter "samples" is a list of
    dictonaries that was aggregated from running a RAG process and collecting the input and output data. Parameter
    "metrics" is a list of metric objects from RAGAS to evaluate the given samples. 
    '''
    
    # Keep track of time
    startTime = time.time()

    # Evaluate every sample
    print("Starting Evaluation...")
    evalResults, metricStats = await pipelineEvaluation(samples, metrics)
    print("Evaluation Finished!")
    for result in evalResults:
        print(result)

    # Compute and output the mean and standard deviation for each metric
    print("+-+-+-+-+-+-+-+-+-+-+-+FINAL RESULTS+-+-+-+-+-+-+-+-+-+-+-+")
    for metric in metricStats.keys():
        print(f"{metric} - Mean: {metricStats[metric]['mean']}, St. Dev: {metricStats[metric]['std_dev']}")
    
    # Elapsed time
    endTime = time.time()
    elapsedTime = endTime - startTime
    if elapsedTime > 60:
        secToMin = elapsedTime / 60
        if secToMin > 60:
            print(f"\nExecution Time: {(secToMin / 60):.2f} hrs")
        else:
            print(f"\nExecution Time: {secToMin:.2f} min")
    
    else:
        print(f"\nExecution Time: {(elapsedTime):.2f} sec")

In [12]:
# Load metrics
evalMetrics = [LLMContextRecall(llm=LangchainLLMWrapper(llm)), 
               FactualCorrectness(llm=LangchainLLMWrapper(llm)), 
               Faithfulness(llm=LangchainLLMWrapper(llm)), 
               SemanticSimilarity(embeddings=LangchainEmbeddingsWrapper(OpenAIEmbeddings())),
               LLMContextPrecisionWithoutReference(llm=LangchainLLMWrapper(llm))
               ]

In [13]:
# Sample records from our dataset
maristTestSample = maristQA.sample(50, replace=False)
maristTestSample.head()

,0,1
470,Who is Desiree Dighton?,Contact InformationAcademic SchoolOfficeEmailP...
359,How can I find out more about the Clinical hea...,Master of Arts in Clinical Mental Health Couns...
546,Who published Displacements and Transformation...,Contact InformationAcademic SchoolOfficeEmailP...
103,What is the number for financial services?,Transfer: Information for Accepted Transfer St...
111,What educational areas does Marist give master...,Master of Arts in Educational Psychology Depar...


In [14]:
# Run our chain with a question and evaluate
for row in maristTestSample.iterrows():
    response = correctiveRAGLangChain(row[1][0], vectorDB, row[1][1])
    print(f"Generation: {response}")
    
print("Generations done!\n")


Processing query: Who is Desiree Dighton?

Retrieved 3 documents
Evaluation scores: [0.1, 0.1, 0.1]

Action: Incorrect - Performing web search


/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


Error parsing search results. Returning empty list.

Final knowledge:
['- Dr. Desiree Dighton is an assistant professor in the English department.', '- A major concern for Dighton is building student confidence in using generative AI tools.', '- Dighton emphasizes the importance of critical thinking and writing skills for students.', '- Ethan Moseley, a BTC certificate student, received the Gold Scholarship from the ECU College of Business.', '- Moseley will start the MA in English program alongside the MBA program in Spring 2025.', '- Dighton and Brent Henze have mentored Moseley and provided feedback on his scholarship materials.', '- Dighton worked with students on designing book reviews for the NCLR winter issue.', '- The winter issue includes over a hundred pages of content, including poetry and creative nonfiction.', "- Dighton praised Sylvia's research and writing on data and society, calling it relevant and approachable for undergraduates."]

Sources:

Generating response...

R

/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


Error parsing search results. Returning empty list.

Final knowledge:
- Title: Transformations in Caribbean Cultures
- Publisher: University Press of Florida
- Publication Date: Spring 2008
- Title: Travelers Possessed: Generic Hybrids and the Caribbean
- Context: Between Anthropology and Literature: Interdisciplinary Discourse
- Publisher: Routledge Press
- Publication Date: 2002
- Pages: 248-266
- Title: El drama del silencio: Renacimiento ritual en la literatura caribena
- Context: Cultural (Con) fusion? Trans-Caribbean Performers and Performance
- Publisher: Caribe 2000 Series
- Publication Date: 2002
- Pages: 49-63
- Title: Women Adrift: Madwomen, Matriarchs, and the Caribbean
- Context: Women at Sea: Travel Writing and the Margins of Caribbean Discourse
- Publisher: Palgrave/St. Martin's Press
- Publication Date: 2001
- Pages: 135-160
- Title: Sorcerers, She-Devils, and Shipwrecked Women: Writing Religion in French Caribbean Literature
- Context: Sacred Possessions: Vodou, Santer

...

Error parsing search results. Returning empty list.

Final knowledge:
['- Rover Environmental Monitoring Station (REMS) data certified from Aug 7, 2012, to July 28, 2024 (ongoing).', '- REMS includes sensors for wind, temperature, humidity, and pressure.', '- Artistic representation of Curiosity rover with REMS Booms and Ultraviolet Sensor.', '- Link to REMS data: https://atmos.nmsu.edu/data_and_services/atmospheres_data/MARS/curiosity/rems.html', '- AirData Air Quality Monitors app available on web and mobile for mapping monitor locations and information.', '- Link to Air Quality Monitors interactive map: https://www.epa.gov/outdoor-air-quality-data/interactive-map-air-quality-monitors', '- Lunar Environment Monitoring Station (LEMS) is part of Artemis III mission to explore lunar South Pole, planned for launch in 2026.', '- Artemis III marks the return of humans to the Moon since Apollo program (1969-1972).', '- Link to LEMS information: https://science.nasa.gov/lems/', '- Environmen

/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


Error parsing search results. Returning empty list.

Final knowledge:
- School of Professional Programs and Graduate & Adult Enrollment established.
- In 1994, a $27 million project completed to create a new Student Center.
- The Student Center features:
- Expansive bookstore
- New dining facilities
- Cabaret
- Dramatic rotunda
- Admissions Office
- A mid-rise residence hall for 382 students in suites was included.
- New offices for Student Affairs and Student Government were part of the project.
- A new set of townhouses for 144 students opened in the same year.
- On-campus graduate housing is more affordable in ultra-urban areas like Chicago, New York, Washington, D.C., and Los Angeles.
- Doctoral students in the Arts and Sciences are eligible for five consecutive academic years of guaranteed student housing if they apply by the deadline, remain registered, and maintain good academic and administrative standing.
- On-campus housing for graduate students at the University at Buffalo i

/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


Error parsing search results. Returning empty list.

Final knowledge:
- Marist College's Strategic Plan covers the period 2018-2023.
- The plan is titled "Student Success, Innovation, and the Social Good."
- Developed through a year-long, broad-based, and inclusive process.
- Aims to build on Marist's successes and enter a new chapter.
- Principles from Marist Brother founders include excellence in education, community, and service.
- Emphasizes the need for innovation and institutional agility in higher education.
- Focuses on three essential goals: Ensuring Student Success, Promoting Innovation, and Advancing the Social Good.
- The FY23-27 Strategic Plan outlines the missions and goals of the U.S. Department of Homeland Security (DHS).
- It fulfills the GPRA Modification Act of 2010 and OMB Circular A-11, Part 6 (2013).
- DHS aims to build a resilient nation capable of withstanding current and future threats.
- Progress was made in the goals set out in the 2022-2026 Strategic Plan by

In [15]:
# Evaluation for LLMContextRecall
await computeEvaluationMetrics(evaluationSamples, [evalMetrics[0]])

Starting Evaluation...
Evaluation Finished!
{'question': 'Who is Desiree Dighton?', 'LLMContextRecall': 0.3333333333333333}
{'question': 'How can I find out more about the Clinical health counseling program?', 'LLMContextRecall': 0.75}
{'question': 'Who published Displacements and Transformations in Caribbean Cultures?', 'LLMContextRecall': 0.25}
{'question': 'What is the number for financial services?', 'LLMContextRecall': 0.5}
{'question': 'What educational areas does Marist give masters in?', 'LLMContextRecall': 0.2}
{'question': 'School of Communications internship opportunities', 'LLMContextRecall': 0.0}
{'question': 'Nicholas Marshall', 'LLMContextRecall': 0.0}
{'question': 'About Fanfarelli', 'LLMContextRecall': 'Error: The LLM generation was not completed. Please increase try increasing the max_tokens and try again.'}
{'question': 'How do a report a covid 19 issue on campus?', 'LLMContextRecall': 0.42857142857142855}
{'question': 'how many distribution credits do I need?', 'LLM

In [16]:
# Evaluation for FactualCorrectness
await computeEvaluationMetrics(evaluationSamples, [evalMetrics[1]])

Starting Evaluation...
Evaluation Finished!
{'question': 'Who is Desiree Dighton?', 'FactualCorrectness': 'Error: The LLM generation was not completed. Please increase try increasing the max_tokens and try again.'}
{'question': 'How can I find out more about the Clinical health counseling program?', 'FactualCorrectness': 0.67}
{'question': 'Who published Displacements and Transformations in Caribbean Cultures?', 'FactualCorrectness': 'Error: The LLM generation was not completed. Please increase try increasing the max_tokens and try again.'}
{'question': 'What is the number for financial services?', 'FactualCorrectness': 'Error: The LLM generation was not completed. Please increase try increasing the max_tokens and try again.'}
{'question': 'What educational areas does Marist give masters in?', 'FactualCorrectness': 'Error: The LLM generation was not completed. Please increase try increasing the max_tokens and try again.'}
{'question': 'School of Communications internship opportunities'

In [17]:
# Evaluation for Faithfulness
await computeEvaluationMetrics(evaluationSamples, [evalMetrics[2]])

Starting Evaluation...
Evaluation Finished!
{'question': 'Who is Desiree Dighton?', 'Faithfulness': 1.0}
{'question': 'How can I find out more about the Clinical health counseling program?', 'Faithfulness': 0.6666666666666666}
{'question': 'Who published Displacements and Transformations in Caribbean Cultures?', 'Faithfulness': 0.25}
{'question': 'What is the number for financial services?', 'Faithfulness': 1.0}
{'question': 'What educational areas does Marist give masters in?', 'Faithfulness': 1.0}
{'question': 'School of Communications internship opportunities', 'Faithfulness': 1.0}
{'question': 'Nicholas Marshall', 'Faithfulness': 1.0}
{'question': 'About Fanfarelli', 'Faithfulness': 0.9333333333333333}
{'question': 'How do a report a covid 19 issue on campus?', 'Faithfulness': 0.7}
{'question': 'how many distribution credits do I need?', 'Faithfulness': 1.0}
{'question': 'Dr. Elizabeth Kaknes?', 'Faithfulness': 0.9090909090909091}
{'question': 'Who is Dr. Lynn Eckert?', 'Faithfulne

In [18]:
# Evaluation for SemanticSimilarity
await computeEvaluationMetrics(evaluationSamples, [evalMetrics[3]])

Starting Evaluation...
Evaluation Finished!
{'question': 'Who is Desiree Dighton?', 'SemanticSimilarity': 0.9210551869163699}
{'question': 'How can I find out more about the Clinical health counseling program?', 'SemanticSimilarity': 0.9442127353057344}
{'question': 'Who published Displacements and Transformations in Caribbean Cultures?', 'SemanticSimilarity': 0.8268826685361922}
{'question': 'What is the number for financial services?', 'SemanticSimilarity': 0.8807925079739356}
{'question': 'What educational areas does Marist give masters in?', 'SemanticSimilarity': 0.8429485289235374}
{'question': 'School of Communications internship opportunities', 'SemanticSimilarity': 0.8966025720243086}
{'question': 'Nicholas Marshall', 'SemanticSimilarity': 0.8072944692808861}
{'question': 'About Fanfarelli', 'SemanticSimilarity': 0.9568380732491247}
{'question': 'How do a report a covid 19 issue on campus?', 'SemanticSimilarity': 0.7606187431012537}
{'question': 'how many distribution credits d

### Full CRAG Results:
- LLMContextRecall - Mean: 0.410, St. Dev: 0.325
- FactualCorrectness - Mean: 0.396, St. Dev: 0.276
- Faithfulness - Mean: 0.824, St. Dev: 0.213
- SemanticSimilarity - Mean: 0.881, St. Dev: 0.065

### Manual Evaluation

In [13]:
# Run our chain with a question and evaluate
for row in maristQA.iterrows():
    response = correctiveRAGLangChain(row[1][0], vectorDB, row[1][1])
    print(f"Generation: {response}")
    
print("Generations done!\n")

/var/folders/6g/8868nrgn5znbjrhdbycnn8pw0000gn/T/ipykernel_4215/3839898041.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  response = correctiveRAGLangChain(row[1][0], vectorDB, row[1][1])



Processing query: What is a Marist ally?

Retrieved 3 documents
Evaluation scores: [0.6, 0.8, 0.9]

Action: Correct - Using retrieved document

Final knowledge:
The Marist Community Department LGBTQ+ Name Marisa Moore Phone 845-575-3314 Location Murray Student Center, 352 Marist Ally NetworkThe \u201cAlly Network\u201d symbol is a message to lesbian, gay, bisexual, transgender, and questioning people and their allies. The message is that the person displaying this symbol promises to be understanding, supportive and trustworthy if a lesbian, gay, bisexual or transgender person needs help, advice or someone with whom s/he can talk. The person displaying this symbol can also provide access to valuable information and resources.Our GoalThe goal of this program is to provide a welcoming environment for lesbian, gay, bisexual or transgender persons by establishing an identifiable network of persons who can provide support, information and a safe place for LGBTQ persons within our campus com

/var/folders/6g/8868nrgn5znbjrhdbycnn8pw0000gn/T/ipykernel_4215/3839898041.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  response = correctiveRAGLangChain(row[1][0], vectorDB, row[1][1])



Retrieved 3 documents
Evaluation scores: [0.5, 0.2, 1.0]

Action: Correct - Using retrieved document

Final knowledge:
Health Services Department Health Services Phone 845-575-3270 Location 69 W. Cedar St. Poughkeepsie, NY Office Hours Monday - Friday8:30 am - 5:00 pm Health ServicesWelcome to Marist from Health Services! Our job is to help keep you healthy and informed. To that end, we work closely with the entire Marist College community as well as the Dutchess County and New York State Departments of Health.There are a lot of changes on campus this year, including where you will find us! In order to meet Reopen guidelines, we have relocated to a different office to ensure the safety of the community. We\u2019ve also had to modify the way we see patients to minimize risk for all involved.Flu shots are mandatory this year and to help students meet this requirement, there will be flu clinics on campus regularly throughout the Fall semester. In addition, students will have ample time b

/var/folders/6g/8868nrgn5znbjrhdbycnn8pw0000gn/T/ipykernel_4215/3839898041.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  response = correctiveRAGLangChain(row[1][0], vectorDB, row[1][1])


...


Retrieved 3 documents
Evaluation scores: [0.6, 0.2, 0.2]

Action: Ambiguous - Combining retrieved document and web search


/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


Error parsing search results. Returning empty list.

Final knowledge:
- Courses include FYS 101 (First Year Seminar) and ENG 120 (Writing for College).
- Students must complete a total of 36 distribution credits to graduate.
- Breadth and Pathway courses can overlap.
- Breadth Requirements: 1 course in each topic.
- Pathway Requirements: 4 courses engaging with an interdisciplinary topic.
- Skill Requirements include intensive courses in Technological Competency and Public Presentation.
- Capping Requirement: Senior-level course in the major.
- Capping courses address academic skills, values awareness, and professional issues related to majors.
- Magna Cum Laude honors require a GPA of 3.750 to 3.949.
- Cum Laude honors require a GPA of 3.500 to 3.749.
- Undergraduate students must earn at least 45 units at California State University, Long Beach to be eligible for honors at graduation.
- Students must complete remaining General Education (GE) breadth and graduation requirements, inclu

/var/folders/6g/8868nrgn5znbjrhdbycnn8pw0000gn/T/ipykernel_4215/3839898041.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  response = correctiveRAGLangChain(row[1][0], vectorDB, row[1][1])



Retrieved 3 documents
Evaluation scores: [0.8, 0.8, 0.7]

Action: Correct - Using retrieved document

Final knowledge:
in studying in the United Kingdom.\u201dAll of the admissions representatives provided brief overviews of their institutions and their offerings before all joined in on a general question-and-answer session. Additional schools on the panel were Keele University and Lancaster University based in England and the University of Strathclyde and the University of Edinburgh in Scotland.Differences between graduate study in the United States and the United Kingdom that came up in the panel discussion included the different lengths of master\u2019s programs in the United Kingdom, with some taking twelve months to complete while others last the two years we are more accustomed to in America. Further, UK graduate study is designated as either research-based---the MRes---or that emphasizing more foundational studies acquired through the taught master\u2019s degree. Students who a

In [14]:
# Evaluation for ContextPrecision
await computeEvaluationMetrics(evaluationSamples, [evalMetrics[4]])

Starting Evaluation...


Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=611a562b-2f98-4f03-bd47-3f1423db8fc3,id=eab39df9-cbfa-4008-a90c-fbd2fa73ef77; trace=879ff678-280c-4ee7-a580-eb3dca49124b,id=879ff678-280c-4ee7-a580-eb3dca49124b; trace=879ff678-280c-4ee7-a580-eb3dca49124b,id=759a86db-cbe4-45a1-8fbd-ad700732bb47; trace=879ff678-280c-4ee7-a580-eb3dca49124b,id=ec865ab7-391b-49e4-b729-f1c35c94b545; trace=879ff678-280c-4ee7-a580-eb3dca49124b,id=691a6c60-786e-4064-93f0-9d8ec6693804; trace=bb5b49be-2c9c-421c-a663-ea1a76b08fcb,id=bb5b49be-2c9c-421c-a663-ea1a76b08fcb; trace=bb5b49be-2c9c-421c-a663-ea1a76b08fcb,id=5a8560ed-bba5-452d-ab45-b349af61d5c8; trace=bb5b49be-2c9c-421c-a663-ea1a76b08fcb,id=62d810d7-5efb-4e56-8c9c-dd553427f4af; trace=bb5b49be-2c9c-421c

Evaluation Finished!
{'question': 'What is a Marist ally?', 'LLMContextPrecisionWithoutReference': 0.9999999999}
{'question': 'Are flu shots required for students?', 'LLMContextPrecisionWithoutReference': 0.9999999999}
{'question': 'Are there career fairs on campus?', 'LLMContextPrecisionWithoutReference': 0.9999999999}
{'question': 'Juan-Manuel Olivera-Silvera about', 'LLMContextPrecisionWithoutReference': 0.9999999999}
{'question': 'Studying philosophy.', 'LLMContextPrecisionWithoutReference': 0.9999999999}
{'question': 'Marist social media', 'LLMContextPrecisionWithoutReference': 0.9999999999}
{'question': 'Who teaches the game of thrones course?', 'LLMContextPrecisionWithoutReference': 0.9999999999}
{'question': 'What is the multicultural affairs center?', 'LLMContextPrecisionWithoutReference': 0.9999999999}
{'question': 'How much may be given in a research grant?', 'LLMContextPrecisionWithoutReference': 0.9999999999}
{'question': 'Janine Larmon Peterson', 'LLMContextPrecisionWitho

Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=e64994ac-3aa2-494d-9824-30b367d94d0d,id=e64994ac-3aa2-494d-9824-30b367d94d0d; trace=e64994ac-3aa2-494d-9824-30b367d94d0d,id=e2bf9981-d743-4827-85c8-c5cf0bf1ff61; trace=e64994ac-3aa2-494d-9824-30b367d94d0d,id=5dfa418e-be64-447c-98f8-4918d14f706e; trace=5caf137d-eff5-4775-8b91-c3843ba1c05b,id=5caf137d-eff5-4775-8b91-c3843ba1c05b; trace=5caf137d-eff5-4775-8b91-c3843ba1c05b,id=f6fa493f-ff42-41dd-9718-9598eb07b151; trace=5caf137d-eff5-4775-8b91-c3843ba1c05b,id=10607b4a-1c66-4cb0-b25b-d1838cb4c5b1; trace=5caf137d-eff5-4775-8b91-c3843ba1c05b,id=55628a86-2fa3-4302-9b41-4eecc43a08dc; trace=cbacf92f-c426-4165-8172-98dccbf474ae,id=cbacf92f-c426-4165-8172-98dccbf474ae; trace=cbacf92f-c426-4165

In [69]:
# Run our chain with a question and evaluate
question = "Who is the chair of the Modern Languages and Cultures Department?"
sample = maristQA.loc[maristQA[0] == question]
for row in sample.iterrows():
    response = correctiveRAGLangChain(row[1][0], vectorDB, row[1][1])
    print(f"Generation: {response}")
    
print("Generations done!\n")


Processing query: Who is the chair of the Modern Languages and Cultures Department?

Retrieved 3 documents
Evaluation scores: [0.2, 0.2, 0.2]

Action: Incorrect - Performing web search


/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


Error parsing search results. Returning empty list.

Final knowledge:
["- RIT's Modern Languages and Cultures department received an International Research and Studies grant from the U.S. Department of Education.", '- The grant will fund the development of new materials and curricula for Chinese, Italian, and Spanish courses focusing on STEAM.', '- Department Chair Sara Armengot will oversee the project for the next three years.', '- Dr. Kelly Suero was named Chair of the World Languages and Cultures Department at Troy University in August 2024.', '- Suero has been with the department since August 2014 and succeeds Dr. Peter Howard, who retired after 50 years.', '- The Department of Modern Languages and Cultures at Rochester University welcomed Russian poet Dmitry Bykov on January 22, 2024.', '- Bykov is a prominent figure in the anti-Putin opposition and is known for his work as a poet, journalist, and professor.', '- Ana Carballal will become the new chair of the Department of World 

In [51]:
# Save as an excel file
samplesOutputDF.to_excel("/Users/christiansarmiento/Library/CloudStorage/OneDrive-MaristCollege/Machine Learning/Data/cRAG_manual_eval.xlsx", index=False)

# Rerun with Anthropic LLM

In [22]:
# LLM chains for retrieval evaluator, knowledge refinement, & query rewriter - redefining here since
# these functions use a global LLM variable

## Load Anthropic LLM
os.environ["ANTHROPIC_API_KEY"] = anthropicKey()
llm = ChatAnthropic(model='claude-3-haiku-20240307', max_tokens_to_sample=1000)

## Retrieval Evaluator
class RetrievalEvaluatorInput(BaseModel):
    relevance_score: float = Field(..., description="The relevance score of the document to the query. the score should be between 0 and 1.")

def retrieval_evaluator(query: str, document: str) -> float:
    prompt = PromptTemplate(
        input_variables=["query", "document"],
        template="On a scale from 0 to 1, how relevant is the following document to the query? Query: {query}\nDocument: {document}\nRelevance score:"
    )
    chain = prompt | llm.with_structured_output(RetrievalEvaluatorInput)
    input_variables = {"query": query, "document": document}
    result = chain.invoke(input_variables).relevance_score
    return result

## Knowledge Refinement
class KnowledgeRefinementInput(BaseModel):
    key_points: str = Field(..., description="The document to extract key information from.")

def knowledge_refinement(document: str) -> List[str]:
    prompt = PromptTemplate(
        input_variables=["document"],
        template="Extract the key information from the following document in bullet points:\n{document}\nKey points:"
    )
    chain = prompt | llm.with_structured_output(KnowledgeRefinementInput)
    input_variables = {"document": document}
    result = chain.invoke(input_variables).key_points
    return [point.strip() for point in result.split('\n') if point.strip()]

## Web Search Query Rewriter
class QueryRewriterInput(BaseModel):
    query: str = Field(..., description="The query to rewrite.")
    
def rewrite_query(query: str) -> str:
    prompt = PromptTemplate(
        input_variables=["query"],
        template="Rewrite the following query to make it more suitable for a web search:\n{query}\nRewritten query:"
    )
    chain = prompt | llm.with_structured_output(QueryRewriterInput)
    input_variables = {"query": query}
    return chain.invoke(input_variables).query.strip()

In [23]:
# Run our chain with a question and evaluate
for row in maristTestSample.iterrows():
    response = correctiveRAGLangChain(row[1][0], vectorDB, row[1][1])
    print(f"Generation: {response}")
    
print("Generations done!\n")


Processing query: Who is Desiree Dighton?

Retrieved 3 documents
Evaluation scores: [0.8, 0.8, 0.7]

Action: Correct - Using retrieved document

Final knowledge:
History and Cultural History) from Indiana University, Bloomington.She has also worked in the business world - for the Outsource Department at IBM, as an office manager and benefits administrator for the National Scholastic Chess Foundation, and as Assistant Director for an academic department in university administration.She loves to tell her students that she obtained those positions with a liberal arts degree of the most seemingly impractical field: an undergraduate major in Medieval Studies and a minor in Classical Languages.Research Interests / Areas of FocusDr. Peterson's research focuses on challenges to the authority of the institutional church, particularly with regard to heresy and saintscults, in late medieval Italy. She is also interested in issues of gender and sexuality, the marginalization of social groups, and

/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


Error parsing search results. Returning empty list.

Final knowledge:
AcademicsAcademicsNo matter your academic interests, Marist has a program of study that is right for you. The Marist educational experience is grounded in the liberal arts, and it focuses on helping students develop the knowledge, character, and skills they'll need to succeed and adapt through a lifetime of change. Here at the College, student success begins with a broad education across many disciplines – no matter their field of study, students learn to think logically and creatively and communicate effectively, both verbally and in writing. Quality teaching in small classes is also key – when professors really know their students, this leads to mentorship, joint research, and intellectual guidance. We also believe that extending learning beyond the classroom is key to intellectual development and critical thinking. Through internships and other partnerships with outside organizations, our students gain real world 

/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


Error parsing search results. Returning empty list.

Final knowledge:
['- The Gallup Resident Agency of the Federal Bureau Investigation investigated the case with assistance from the Navajo Police Department, Department of Criminal Investigation, New Mexico State Police, and Gallup Police Department.', '- Assistant United States Attorney Nicholas Marshall prosecuted the case.', '- In a sentencing memorandum, Marshall wrote that Torrez was "arguably the second most culpable", behind her mother, in a 2018 killing.', '- Torrez was interrogated by investigators but never admitted to any involvement in the 2018 killing.', '- Marshall also agreed in a plea deal to not bring charges for an additional crime.', '- Nicholas Marshall is a second-year architecture major and director of student organizations at NJIT.']

Sources:

Generating response...

Response generated
Generation: Based on the provided knowledge, there are two individuals named Nicholas Marshall:

1. Nicholas Marshall, the Assi

...


Response generated
Generation: Based on the provided knowledge, the school offers housing options for graduate students. The key information is:

1. In 1994, a new mid-rise residence hall housing 382 students in suites was constructed as part of a $27 million project to create a new Student Center.
2. Additionally, a new set of townhouses for 144 students also opened that same year.

These housing options appear to be available for graduate students, in addition to the undergraduate population.

Source:
The information is retrieved from an unspecified document, as indicated in the "Sources: Retrieved document" section.

Processing query: What is the goal of the 2023 strategic plan?


Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=b60d51c1-7948-4c18-b2cd-35e39dcb8df9,id=d24a3618-25c0-4841-b264-fb085f6b184e; trace=7192c863-f672-492d-ad97-340e05f2d77b,id=7192c863-f672-492d-ad97-340e05f2d77b; trace=7192c863-f672-492d-ad97-340e05f2d77b,id=e6a45fc5-8c9d-45a2-b849-3375a66b28d9; trace=7192c863-f672-492d-ad97-340e05f2d77b,id=f8582c20-6349-442e-b7d2-32360b33a537; trace=7192c863-f672-492d-ad97-340e05f2d77b,id=2522836d-1605-4574-b965-51ee8774b8a0; trace=94af5f8f-b9de-40ea-a202-46898e113ca6,id=94af5f8f-b9de-40ea-a202-46898e113ca6; trace=94af5f8f-b9de-40ea-a202-46898e113ca6,id=b641696c-89d7-402b-bb58-e6283081e3c6; trace=94af5f8f-b9de-40ea-a202-46898e113ca6,id=f8f60ff5-13bf-4058-8c95-bff163af9135; trace=7e31820c-901c-4453


Retrieved 3 documents
Evaluation scores: [0.7, 0.7, 0.7]

Action: Ambiguous - Combining retrieved document and web search


/Users/christiansarmiento/opt/anaconda3/envs/LLM-LangChain/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


Error parsing search results. Returning empty list.

Final knowledge:
About Marist College Strategic Plan 2018-2023
- Developed through a year-long, broad-based and inclusive process
- Ambitious but achievable plan to build on Marist's successes and take the college into a new chapter
- Guided by Marist's founding principles of excellence in education, sense of community, and commitment to service
- Recognizes the need to embrace innovation and institutional agility to thrive in a rapidly changing higher education environment
- Focuses on 3 essential goals:
1. Ensuring Student Success
2. Promoting Innovation
3. Advancing the Social Good
- These goals and supporting initiatives will assist in setting priorities for the next 5 years
- The FY23-27 Strategic Plan articulates the Department of Homeland Security's missions and goals, the strategies to achieve each goal, and long-term performance measures to evaluate progress.
- The Strategic Plan fulfills the GPRA Modification Act of 2010 an

In [24]:
# Evaluation for LLMContextRecall
await computeEvaluationMetrics(evaluationSamples, [evalMetrics[0]])

Starting Evaluation...


Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=c38fa9da-0d27-490d-a522-63a05b87e615,id=670cd5ee-c917-4703-aaa3-8430472bac0d; trace=c38fa9da-0d27-490d-a522-63a05b87e615,id=7e779de6-9b1b-491e-b0b3-75511af0f3ff; trace=6ca04896-3b17-47c6-88fe-9bbf8071d5e8,id=6ca04896-3b17-47c6-88fe-9bbf8071d5e8; trace=6ca04896-3b17-47c6-88fe-9bbf8071d5e8,id=7e03c55d-d93d-4398-bb6f-ad9b301edbbb; trace=6ca04896-3b17-47c6-88fe-9bbf8071d5e8,id=6b1ce942-79bd-4fc1-a0be-2859acac144e; trace=6ca04896-3b17-47c6-88fe-9bbf8071d5e8,id=51ff1816-e83d-4060-8ea4-cd4e19ade880; trace=cec3dbfb-0078-4f2c-863c-1461d75eb2b9,id=cec3dbfb-0078-4f2c-863c-1461d75eb2b9; trace=201562b5-dd13-4344-bab7-3e216f9eb9fd,id=201562b5-dd13-4344-bab7-3e216f9eb9fd; trace=201562b5-dd13-4344

Evaluation Finished!
{'question': 'Who is Desiree Dighton?', 'LLMContextRecall': 0.6}
{'question': 'How can I find out more about the Clinical health counseling program?', 'LLMContextRecall': 0.8}
{'question': 'Who published Displacements and Transformations in Caribbean Cultures?', 'LLMContextRecall': 0.5}
{'question': 'What is the number for financial services?', 'LLMContextRecall': 0.3333333333333333}
{'question': 'What educational areas does Marist give masters in?', 'LLMContextRecall': 0.2}
{'question': 'School of Communications internship opportunities', 'LLMContextRecall': 0.5}
{'question': 'Nicholas Marshall', 'LLMContextRecall': 0.0}
{'question': 'About Fanfarelli', 'LLMContextRecall': 0.5454545454545454}
{'question': 'How do a report a covid 19 issue on campus?', 'LLMContextRecall': 0.7272727272727273}
{'question': 'how many distribution credits do I need?', 'LLMContextRecall': 0.5}
{'question': 'Dr. Elizabeth Kaknes?', 'LLMContextRecall': 0.75}
{'question': 'Who is Dr. Lynn 

In [25]:
# Evaluation for FactualCorrectness
await computeEvaluationMetrics(evaluationSamples, [evalMetrics[1]])

Starting Evaluation...


Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=cfa28c24-7dc4-4c9a-824b-1fec57188089,id=cfa28c24-7dc4-4c9a-824b-1fec57188089; trace=cfa28c24-7dc4-4c9a-824b-1fec57188089,id=23b166f0-7c1c-49b9-baca-0a4e1f2327e9; trace=cfa28c24-7dc4-4c9a-824b-1fec57188089,id=91ca0a50-8f4d-43d6-a3b6-7f4b5914350d; trace=d9db29c1-7110-4392-89d6-d10b64624d65,id=d9db29c1-7110-4392-89d6-d10b64624d65; trace=d9db29c1-7110-4392-89d6-d10b64624d65,id=3e981794-6529-469e-866b-b15ad0f4fc68; trace=d9db29c1-7110-4392-89d6-d10b64624d65,id=4b6a178c-d117-4365-b323-652b3acca211; trace=fdc97b65-8a4c-4a78-9adc-74e5b5eed62f,id=fdc97b65-8a4c-4a78-9adc-74e5b5eed62f; trace=fdc97b65-8a4c-4a78-9adc-74e5b5eed62f,id=02a80c5a-93fe-4eba-9989-7fe71c68983d; trace=fdc97b65-8a4c-4a78

Evaluation Finished!
{'question': 'Who is Desiree Dighton?', 'FactualCorrectness': 'Error: The LLM generation was not completed. Please increase try increasing the max_tokens and try again.'}
{'question': 'How can I find out more about the Clinical health counseling program?', 'FactualCorrectness': 0.67}
{'question': 'Who published Displacements and Transformations in Caribbean Cultures?', 'FactualCorrectness': 'Error: The LLM generation was not completed. Please increase try increasing the max_tokens and try again.'}
{'question': 'What is the number for financial services?', 'FactualCorrectness': 'Error: The LLM generation was not completed. Please increase try increasing the max_tokens and try again.'}
{'question': 'What educational areas does Marist give masters in?', 'FactualCorrectness': 'Error: The LLM generation was not completed. Please increase try increasing the max_tokens and try again.'}
{'question': 'School of Communications internship opportunities', 'FactualCorrectness':

In [28]:
# Evaluation for Faithfulness
await computeEvaluationMetrics(evaluationSamples, [evalMetrics[2]])

Starting Evaluation...


No statements were generated from the answer.


Evaluation Finished!
{'question': 'Who is Desiree Dighton?', 'Faithfulness': 1.0}
{'question': 'How can I find out more about the Clinical health counseling program?', 'Faithfulness': 0.5555555555555556}
{'question': 'Who published Displacements and Transformations in Caribbean Cultures?', 'Faithfulness': 0.25}
{'question': 'What is the number for financial services?', 'Faithfulness': 1.0}
{'question': 'What educational areas does Marist give masters in?', 'Faithfulness': 1.0}
{'question': 'School of Communications internship opportunities', 'Faithfulness': 0.8823529411764706}
{'question': 'Nicholas Marshall', 'Faithfulness': 1.0}
{'question': 'About Fanfarelli', 'Faithfulness': 'Error: The LLM generation was not completed. Please increase try increasing the max_tokens and try again.'}
{'question': 'How do a report a covid 19 issue on campus?', 'Faithfulness': 0.7}
{'question': 'how many distribution credits do I need?', 'Faithfulness': 1.0}
{'question': 'Dr. Elizabeth Kaknes?', 'Faith

In [27]:
# Evaluation for SemanticSimilarity
await computeEvaluationMetrics(evaluationSamples, [evalMetrics[3]])

Starting Evaluation...


Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=ee7fb091-40d2-4dae-84e2-def5aa6b3844,id=0764fb98-4bfb-4160-979c-07a9d27c38c8; trace=ee7fb091-40d2-4dae-84e2-def5aa6b3844,id=e63392eb-e0d9-44ef-8251-6c33b4928e26; trace=1d3b37e5-28b0-4f54-af33-0d85eff7d0b8,id=1d3b37e5-28b0-4f54-af33-0d85eff7d0b8; trace=1d3b37e5-28b0-4f54-af33-0d85eff7d0b8,id=b3a66b87-2b55-4aff-95b6-8ba8207fea36; trace=1d3b37e5-28b0-4f54-af33-0d85eff7d0b8,id=251c273d-fd2c-40fe-8645-41189d015e71; trace=1d3b37e5-28b0-4f54-af33-0d85eff7d0b8,id=75bfe79f-2c47-450c-a0f6-8a2ebe1480af; trace=1d3b37e5-28b0-4f54-af33-0d85eff7d0b8,id=a6b897a3-ce28-45c8-b357-c1d1ab546df0; trace=d61906ec-c33e-4ae8-9626-9a3b344ef916,id=d61906ec-c33e-4ae8-9626-9a3b344ef916; trace=d61906ec-c33e-4ae8

Evaluation Finished!
{'question': 'Who is Desiree Dighton?', 'SemanticSimilarity': 0.9210551869163699}
{'question': 'How can I find out more about the Clinical health counseling program?', 'SemanticSimilarity': 0.9442127353057344}
{'question': 'Who published Displacements and Transformations in Caribbean Cultures?', 'SemanticSimilarity': 0.8268826685361922}
{'question': 'What is the number for financial services?', 'SemanticSimilarity': 0.8807925079739356}
{'question': 'What educational areas does Marist give masters in?', 'SemanticSimilarity': 0.8429485289235374}
{'question': 'School of Communications internship opportunities', 'SemanticSimilarity': 0.8966025720243086}
{'question': 'Nicholas Marshall', 'SemanticSimilarity': 0.8072944692808861}
{'question': 'About Fanfarelli', 'SemanticSimilarity': 0.9568380732491247}
{'question': 'How do a report a covid 19 issue on campus?', 'SemanticSimilarity': 0.7606187431012537}
{'question': 'how many distribution credits do I need?', 'SemanticSi

Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=2f41f0d3-3bf6-4461-947b-96548149d428,id=2f41f0d3-3bf6-4461-947b-96548149d428; trace=2f41f0d3-3bf6-4461-947b-96548149d428,id=71254d14-d2a5-4fd0-a0b3-e67773009de7; trace=2f41f0d3-3bf6-4461-947b-96548149d428,id=67a1d655-aa50-4919-b44b-543f1f521b60; trace=2f41f0d3-3bf6-4461-947b-96548149d428,id=706d6a14-c9d5-4c34-a449-11cc582feda7; trace=2f41f0d3-3bf6-4461-947b-96548149d428,id=aba7fb96-e6a7-411e-af34-ccb8d8b32a1f; trace=fb6a6609-4514-4cb0-b35f-d6c41ab9af49,id=fb6a6609-4514-4cb0-b35f-d6c41ab9af49; trace=fb6a6609-4514-4cb0-b35f-d6c41ab9af49,id=dbb73f25-7294-426b-9720-d8d4f542533c; trace=fb6a6609-4514-4cb0-b35f-d6c41ab9af49,id=fa268311-f14b-41d7-9715-77f8d8452b75; trace=fb6a6609-4514-4cb0